In [30]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [31]:
df = pd.read_csv('spam_ham_dataset (1).csv')
df.head()

,label,text
0,ham,enron methanol ; meter # : 988291\r\nthis is ...
1,ham,"hpl nom for january 9 , 2001\r\n( see attache..."
2,ham,"neon retreat\r\nho ho ho , we ' re around to ..."
3,spam,"photoshop , windows , office . cheap . main t..."
4,ham,re : indian springs\r\nthis deal is to book t...


In [33]:
df['target'] = df['label'].map( {'spam':1, 'ham':0 })

In [34]:
## b) Trainings- und Testdaten separieren
df_train = df.sample(frac=.8, random_state=11)
df_test = df.drop(df_train.index)
print(df_train.shape, df_test.shape)

(4137, 3) (1034, 3)


In [35]:
y_train = df_train['target'].values
y_test = df_test['target'].values
y_test.shape

(1034,)

In [36]:
X_train = df_train['text'].values
X_test = df_test['text'].values

In [37]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_dict = tokenizer.index_word
#print(len(word_dict))
#print(word_dict)

#for key in word_dict.keys():
#    print(key, word_dict[key])

In [38]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
print(X_train_seq[:5])
print(df_train.iloc[0,:])
for el in X_train_seq[0]:
    print(word_dict[el], end=' ')


[[46, 150, 6, 250, 129, 90, 92, 100, 168, 610, 4274, 118, 1, 610, 4274, 118], [8, 4275, 1872, 14, 698, 22750, 22751, 274, 147, 3, 18, 4557], [3505, 6244, 3680, 264, 4, 3852, 454, 130, 7479, 79, 35, 295, 356, 58, 17, 13, 175, 83, 90, 51, 67, 78, 1, 439, 1586, 13, 175, 83, 28, 125, 906, 268, 91, 3, 1, 45, 1, 37, 3505, 6244, 3680, 264, 4, 3852, 454, 130, 7479, 4558, 1, 191, 52, 1873, 283, 291, 20, 29, 2143, 3, 529, 3505, 6244, 3680, 264, 136, 3852, 454, 130, 282, 3, 2, 4559, 8409, 13, 2, 718, 9685, 15, 1181, 786, 60, 152, 24, 1587, 26, 223, 7, 2, 137, 2434, 392, 2, 8409, 14, 7480, 1, 26, 19, 109, 16, 24, 22752, 2, 320, 4, 1452, 10, 7, 54, 320, 452, 19, 109, 1, 63, 1, 439], [22753, 56, 16, 157, 1947, 3, 237, 4, 245, 386, 19, 111, 1987, 2, 1835, 14961, 6, 2, 540, 19, 10, 233, 1050, 285, 2, 22754, 3853, 9686, 22755, 14, 9687, 22756, 14962, 22757, 22758, 22759, 96, 81, 57, 352, 285, 2, 14961, 85, 20, 69, 3, 11580, 96, 540, 36, 85, 16, 157, 5127, 184, 11, 657, 94, 43, 65, 56, 16, 2435, 3, 31, 

In [39]:
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')
X_train_pad[:5]
X_train_pad.shape

(4137, 20)

In [40]:
laenge_pads = 20
anz_woerter = 7982

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=anz_woerter+1, output_dim=20, input_length=laenge_pads))
lstm_model.add(LSTM(400))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 20)            159660    
                                                                 
 lstm_1 (LSTM)               (None, 400)               673600    
                                                                 
 dense_1 (Dense)             (None, 1)                 401       
                                                                 
Total params: 833,661
Trainable params: 833,661
Non-trainable params: 0
_________________________________________________________________


In [41]:
history = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/embedding_1/embedding_lookup' defined at (most recent call last):
    File "D:\Apps\anaconda\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "D:\Apps\anaconda\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\Apps\anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "D:\Apps\anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "D:\Apps\anaconda\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "D:\Apps\anaconda\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "D:\Apps\anaconda\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
    File "D:\Apps\anaconda\lib\asyncio\

In [15]:
sms_test = ['Hi Paul, would you come around tonight']
sms_seq = tokenizer.texts_to_sequences(sms_test)

sms_pad = pad_sequences(sms_seq, maxlen=20, padding='post')
tokenizer.index_word
sms_pad
y_pred1 = lstm_model.predict(sms_pad)
classes = np.argmax(y_pred1)
classes

1/1 [==============================] - 0s 19ms/step


0

In [17]:
sms_test = ['Free SMS service for anyone']
sms_seq = tokenizer.texts_to_sequences(sms_test)

sms_pad = pad_sequences(sms_seq, maxlen=20, padding='post')
tokenizer.index_word
sms_pad
#lstm_model.predict_classes(sms_pad)
y_pred1 = lstm_model.predict(sms_pad)
classes = np.argmax(y_pred1)
classes

1/1 [==============================] - 0s 17ms/step


0

In [25]:
from sklearn.metrics import accuracy_score

In [21]:
y_red1 = lstm_model.predict(X_test_pad)

35/35 [==============================] - 1s 22ms/step


In [28]:
classes = np.argmax(y_red1,axis = 1)
classes

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [29]:
accuracy_score(y_test,classes)

0.8644524236983842